# Assignment 3 - Part 1

In [29]:
import urllib
import requests
import json
import csv


In [30]:
API = "http://gustav1.ux.uis.no:5002"

Issuing a search query againt the API

In [31]:
def search(indexname, query, field, size=10):
    url = "/".join([API, indexname, "_search"]) + "?" \
          + urllib.parse.urlencode({"q": query, "df": field, "size": size})
    response = requests.get(url).text
    return json.loads(response)

Printing results for a given search query

In [32]:
# res = search("clueweb12b", "united states", "content", size=5)
# for r in res.get("hits", {}).get("hits", {}):
#     print("{} {}".format(r["_id"], r["_score"]))

## TODOs:

- Retrieve results for all queries (in `data/queries.txt`)
- Do it for both `title` and `content` fields (separately) and write the output to files
- Evaluate the results against the relevance judgments (in `data/qrels.csv`) in terms of NDCG@10 and NDCG@20

In [33]:
QUERY_FILE = "data/queries.txt" 
OUTPUT_FILE = "data/baseline_title.txt"
OUTPUT_FILE_2 = "data/baseline_content.txt"
QRELS = "data/qrels.csv"

In [34]:
def load_queries(filename):
    with open(filename, 'rt') as fin:
        queries = {}
        for line in fin.readlines():
            qid, query = line.strip().split(" ", 1)
            queries[qid] = query
    return queries    

In [35]:
load_queries(QUERY_FILE)

{'201': 'raspberry pi',
 '202': 'uss carl vinson',
 '203': 'reviews of les miserables',
 '204': 'rules of golf',
 '205': 'average charitable donation',
 '206': 'wind power',
 '207': 'bph treatment',
 '208': 'doctor zhivago',
 '209': 'land surveyor',
 '210': 'golf gps',
 '211': 'what is madagascar known for',
 '212': 'home theater systems',
 '213': 'carpal tunnel syndrome',
 '214': 'capital gains tax rate',
 '215': 'maryland department of natural resources',
 '216': 'nicolas cage movies',
 '217': 'kids earth day activities',
 '218': 'solar water fountains',
 '219': "what was the name of elvis presley's home",
 '220': 'nba records',
 '221': 'electoral college 2008 results',
 '222': 'male menopause',
 '223': 'usda food pyramid',
 '224': 'making chicken soup from scratch',
 '225': 'black and gold',
 '226': 'traverse city',
 '227': 'i will survive lyrics',
 '228': 'hawaiian volcano observatories',
 '229': 'beef stroganoff recipe',
 '230': "world's biggest dog",
 '231': 'what are the seven d

In [36]:
def load_qrels(filename):
    queries = {}
    with open(filename, 'rt') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',')
        for i, row in enumerate(csvreader):
            if i >0:
                qid = row[0]
                doc_id = row[1]
                rel = row[2]
                queries[qid] = {doc_id: rel}
    return queries

In [37]:
gtruth=load_qrels(QRELS)

In [52]:
def write_file(filename, field):
    f = open(filename, 'w')
    f.write('QueryId,DocumentId\n')  
    queries= {}
    for q_id, query in load_queries(QUERY_FILE).items():
        res = search("clueweb12b", query, field, size=100)
        for r in res.get("hits", {}).get("hits", {}):
#             print(str(q_id) + "," + r["_id"]+ '\n')
            queries[str(q_id)] = r["_id"]
            f.write(str(q_id) + "," + r["_id"]+ '\n')
    f.close() 
    return queries

In [53]:
title=write_file(OUTPUT_FILE, "title")
print(title)

{'201': 'clueweb12-0908wb-98-10600', '202': 'clueweb12-0109wb-05-28625', '203': 'clueweb12-0201wb-40-00755', '204': 'clueweb12-1102wb-77-16467', '205': 'clueweb12-0906wb-61-00511', '206': 'clueweb12-0303wb-55-01304', '207': 'clueweb12-1901wb-11-26808', '208': 'clueweb12-1003wb-80-10571', '209': 'clueweb12-0809wb-12-27431', '210': 'clueweb12-1303wb-14-01207', '211': 'clueweb12-0703wb-94-04778', '212': 'clueweb12-1806wb-97-27816', '213': 'clueweb12-1512wb-49-06549', '214': 'clueweb12-1100tw-78-02859', '215': 'clueweb12-0311wb-99-08583', '216': 'clueweb12-0500wb-02-05873', '217': 'clueweb12-1514wb-56-13231', '218': 'clueweb12-1910wb-15-06592', '219': 'clueweb12-0506wb-42-26106', '220': 'clueweb12-0816wb-30-13266', '221': 'clueweb12-1203wb-38-40639', '222': 'clueweb12-0008wb-53-15013', '223': 'clueweb12-1306wb-87-10712', '224': 'clueweb12-0106wb-01-13792', '225': 'clueweb12-0102wb-07-26372', '226': 'clueweb12-0203wb-03-16678', '227': 'clueweb12-1402wb-46-25430', '228': 'clueweb12-1907wb-54

In [40]:
write_file(OUTPUT_FILE_2, "content")

In [41]:
def dcg(rel, p):
    dcg = rel[0]
    for i in range(1, min(p, len(rel))): 
        dcg += rel[i] / math.log(i + 1, 2)  # rank position is indexed from 1..
    return dcg

    

In [42]:
sum_ndcg10 = 0
sum_ndcg20 = 0

for qid, query in sorted(load_queries(QUERY_FILE).items()):
    gt = gtruth[qid]    
    print("Query", qid)
    
    gains = [] # holds corresponding relevance levels for the ranked docs
    for doc_id in query: 
        gain = gt.get(doc_id, 0)
        gains.append(gain)
    print("\tGains:", gains)
    
    # relevance levels of the idealized ranking
    gain_ideal = sorted([v for _, v in gt.items()], reverse=True)
    print("\tIdeal gains:", gain_ideal)
    
    ndcg5 = dcg(gains, 5) / dcg(gain_ideal, 5)
    ndcg10 = dcg(gains, 10) / dcg(gain_ideal, 10)
    sum_ndcg5 += ndcg5
    sum_ndcg10 += ndcg10
    
    print("\tNDCG@5:", round(ndcg5, 3), "\n\tNDCG@10:", round(ndcg10, 3))

print("Average")
print("\tNDCG@5:", round(sum_ndcg5 / len(rankings), 3), "\n\tNDCG@10:", round(sum_ndcg10 / len(rankings), 3))

Query 201
	Gains: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	Ideal gains: ['1']


NameError: name 'math' is not defined